In [ ]:
import numpy as np 
import os
import cv2
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from keras.preprocessing import image
import gdown
from sklearn.metrics import confusion_matrix, classification_report
import pandas as pd
import seaborn as sns

In [ ]:
file_path = "test.zip"
output = file_path

if not os.path.exists("Face Emotions Images/"):
    gdown.download("https://drive.google.com/uc?id=1QK-WIVObJWHtdQfoigVCpbaY6YYaZIPk", file_path)
    gdown.extractall(file_path)
    if os.path.exists(file_path):
        os.remove(file_path)

In [ ]:
#Data augmentation for disgust category
#import imutils

#db_path = "Face Emotions Images/disgust/"
#for n in range(len(disgust)):
#    cv2.imwrite("C:/Users/Nicho/OneDrive/Desktop/UNI/MAGISTRALE/Visione Artificiale/FER-and-GR-Project/Face Emotions Images/disgust/d{}.png".format(n),cv2.flip(disgust[n],1))

#for n in range(len(disgust)):
#    cv2.imwrite("C:/Users/Nicho/OneDrive/Desktop/UNI/MAGISTRALE/Visione Artificiale/FER-and-GR-Project/Face Emotions Images/disgust/rl{}.png".format(n),imutils.rotate(disgust[10], angle=20))

#for n in range(len(disgust)):
#    cv2.imwrite("C:/Users/Nicho/OneDrive/Desktop/UNI/MAGISTRALE/Visione Artificiale/FER-and-GR-Project/Face Emotions Images/disgust/rr{}.png".format(n),imutils.rotate(disgust[10], angle=-20))


In [ ]:
db_path = "Face Emotions Images/"
batch_size = 64
num_classes = 7
img_height = 224
img_width = 224

In [ ]:
train_ds = tf.keras.preprocessing.image_dataset_from_directory(db_path,
                                                               validation_split=0.20,
                                                               subset="training",
                                                               seed=123456,
                                                               label_mode = 'int',
                                                               color_mode='grayscale',
                                                               image_size=(img_height, img_width),
                                                               batch_size=batch_size)

val_ds = tf.keras.preprocessing.image_dataset_from_directory(db_path,
                                                               validation_split=0.20,
                                                               subset="validation",
                                                               seed=123456,
                                                               label_mode = 'int',
                                                               color_mode='grayscale',
                                                               image_size=(img_height, img_width),
                                                               batch_size=batch_size)


In [ ]:
class_names = train_ds.class_names
plt.figure(figsize=(10, 10))
for images, labels in train_ds.take(1):
    for i in range(9):
        ax = plt.subplot(3, 3, i + 1)
        ax.imshow(images[i].numpy().astype("uint8")[:,:,0], cmap='gray')
        plt.title(class_names[labels[i]])
        plt.axis("off")

In [ ]:
# Build the Model

model = Sequential([
                  layers.experimental.preprocessing.Rescaling(1./255, input_shape=(img_height, img_width,1)),
                  layers.Conv2D(32, 3, padding='same', activation='relu'),
                  layers.MaxPooling2D(),
                  layers.Conv2D(64, 3, padding='same', activation='relu'),
                  layers.MaxPooling2D(),
                  layers.Conv2D(128, 3, padding='same', activation='relu'),
                  layers.MaxPooling2D(),
                  layers.Dropout(0.2),
                  layers.Conv2D(256, 3, padding='same', activation='relu'),
                  layers.MaxPooling2D(),
                  layers.Dropout(0.2),
                  layers.Conv2D(512, 3, padding='same', activation='relu'),
                  layers.MaxPooling2D(),
                  layers.Dropout(0.2),
                  layers.Flatten(),
                  layers.Dense(256, activation='relu'),
                  layers.Dropout(0.2),  
                  layers.Dense(128, activation = 'relu'),
                  layers.Dropout(0.2),
                  layers.Dense(num_classes, activation = 'sigmoid')
])

model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(),
              metrics=['accuracy'])

early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss',
                                                  patience=5,
                                                  mode='auto',
                                                  restore_best_weights=True
                                                 )
model.summary()

In [ ]:
epochs = 50
history = model.fit(train_ds, validation_data = val_ds, epochs = epochs, batch_size = batch_size, 
                    verbose = 2, callbacks=[early_stopping])

In [ ]:
model_acc = model.evaluate(val_ds)[1]
model_acc

In [ ]:
model.save("Emotions Model")

In [ ]:
#Load the Gender Model
if not os.path.exists("Gender Model/"):
    gdown.download("https://drive.google.com/uc?id=1easVnhBN9o1s60_eAyl8CGj_LHuPoXgc", file_path)
    gdown.extractall(file_path)
    if os.path.exists(file_path):
        os.remove(file_path)
gender_model = keras.models.load_model('Gender Model/')

emotions_model = keras.models.load_model('Emotions Model/')
testData = tf.keras.preprocessing.image_dataset_from_directory(
    "Test Set/Emotions",
    labels='inferred',
    label_mode='categorical',
    color_mode='grayscale',
    seed=123456,
    image_size=(img_height,img_width),
    batch_size=1000)


predictions = np.array([])
labels =  np.array([])
for x, y in testData:
    predictions = np.concatenate([predictions, np.argmax(emotions_model.predict(x), axis = -1)])
    labels = np.concatenate([labels, np.argmax(y.numpy(), axis=-1)])

In [ ]:
cm_data = confusion_matrix(labels , predictions)
cm = pd.DataFrame(cm_data, columns=class_names, index = class_names)
cm.index.name = 'Actual'
cm.columns.name = 'Predicted'
plt.figure(figsize = (20,10))
plt.title('Confusion Matrix', fontsize = 20)
sns.set(font_scale=1.2)
ax = sns.heatmap(cm, cbar=False, cmap="Blues", annot=True, annot_kws={"size": 16}, fmt='g')

In [ ]:
print(classification_report(labels, predictions))